# 要約 
このJupyterノートブックは、「Llama-3 8b」を使用してTPU上で大規模言語モデル（LLM）のトレーニングを実行することを目的としています。コンペティションの文脈では、異なるLLMの応答の好みを予測するために必要なデータ処理やモデルのトレーニング手法に焦点を当てています。

### 主な内容と手法:
1. **ライブラリのインポート**: 
   - トレーニングには、`transformers`, `torch`, `peft`, `torch_xla`などのライブラリが使用されます。これらは、MLモデルの構築、データ処理、およびTPUでの効率的なトレーニングに役立ちます。
  
2. **TPU環境の設定**:
   - TPU上での作業を行うため、TPU関連のモジュールがインポートされ、デバイスを初期化しています。

3. **データの前処理**:
   - KaggleのデータセットをCSVから読み込み、プロンプトと応答を整形します。無効なデータ（両方の応答が`null`の場合）は削除され、トークン長が計算されてデータフレームに追加されます。

4. **トークナイゼーション**:
   - トークン化に`AutoTokenizer`を利用し、テキストの長さを設定、パディングを施し、ラベルを取得します。

5. **モデルの構築**:
   - `LlamaForSequenceClassification`を使用して、3つのターゲットラベルに基づく分類モデルを構築します。また、LoRA（Low-Rank Adaptation）の手法を用いてモデルのパラメータ最適化を図ります。

6. **トレーニング実行**:
   - モデルを指定されたエポックでトレーニングし、損失関数として交差エントロピーを使用。学習率の調整やメトリクスの記録を行いながら、各エポックの進捗を追うことができます。

7. **モデルの保存**:
   - トレーニング後、モデルとオプティマイザの状態を保存し、後の利用や再トレーニングに備えます。

8. **ビジュアライゼーション**:
   - トレーニング過程での損失をプロットし、モデルの学習の進捗を評価する手段を提供しています。

### 結論:
このノートブックは、TPUを使用したLLMトレーニングに特化しており、効率的なデータ処理、モデル構築、トレーニング手法を整備しています。トレーニングのスピードや最適化の余地があることを指摘し、さらなる改善のための方向性を示しています。

---


# 用語概説 
以下に、Jupyter Notebookの内容に関連する、機械学習・深層学習の初心者がつまずきそうな専門用語の解説を示します。

1. **TPU (Tensor Processing Unit)**:
   - Googleが開発した専用のハードウェアで、深層学習モデルのトレーニングを加速するために設計されています。GPUよりも高速で大規模なデータ処理に適しているため、大規模なモデルのトレーニングに広く使用されます。

2. **LoRA (Low-Rank Adaptation)**:
   - モデル微調整の手法で、特定のタスクにおいて大規模言語モデルの一部のパラメータを効率的に適応させるためのアプローチです。従来の微調整と比べてメモリ消費を抑え、より少ないデータで効果的に学習を行えます。

3. **シャーディング (Sharding)**:
   - データやモデルの大規模な分散処理の手法で、計算資源を効率的に使用するために、データを複数の部分に分けて同時に処理する方法です。TPU環境では、モデルやデータを複数のTPUデバイスに分散させることで、トレーニング速度を向上させます。

4. **アテンションマスク (Attention Mask)**:
   - モデルが入力を処理する際に、どの部分に注意を向けるべきかを示すための配列です。特に、パディングされた部分を無視するために使用され、計算の効率を高めます。

5. **ウォームアップ (Warmup)**:
   - 学習率スケジュールの一部で、トレーニングの初期段階で学習率を徐々に増加させる手法です。これにより、モデルが安定した状態で学習を開始でき、オーバーシュートを防ぐ効果があります。

6. **ガーベジコレクション (Garbage Collection)**:
   - プログラミングにおいて、自動でメモリを管理し、使用されていないメモリを解放するプロセスです。特に、大規模なデータを扱う機械学習において、メモリ使用量を適切に管理することは重要です。

7. **ロジット (Logits)**:
   - モデルの出力として得られる未正規化の値で、最終的なクラス予測を行う前に、Softmax関数を通じて確率に変換される値です。

8. **SPMD (Single Program Multiple Data)**:
   - 同じプログラムを複数のデータに対して同時に実行するパラダイムです。TPUやGPUでの並列アルゴリズムにおいて、効率的な計算を行うために重要です。

9. **NumPyとPandas**:
   - NumPyは数値計算用のライブラリで、行列演算に特化しており、Pandasはデータ操作のためのライブラリです。特にデータフレームを扱うことで、データの前処理や分析を効率的に行います。

10. **スケジューラ (Scheduler)**:
    - 学習率などのハイパーパラメータをトレーニングの進行に応じて調整するための仕組みです。これにより、トレーニングの各段階で最適な値を使用して、学習の効率を改善します。

この用語集は、該当のノートブックの内容に特有の概念や技術に焦点を当てており、初心者が理解を深めることに役立つでしょう。

---


# Llama-3 8b [TPUトレーニング]

TPU上での大規模言語モデル（LLM）のトレーニングを学びます。これがあなたにも役に立つことを願っています！

このノートブックは以下のリソースに触発されました：

* [LLM detect AI competition Mistral-7B](https://www.kaggle.com/code/hotchpotch/train-llm-detect-ai-comp-mistral-7b/notebook)
* [DAIGT Mistral-7B TPU BFloat16 [トレーニング]](https://www.kaggle.com/code/markwijkhuizen/daigt-mistral-7b-tpu-bfloat16-train)
* [LLAMA 2 13B on TPU (トレーニング)](https://www.kaggle.com/code/defdet/llama-2-13b-on-tpu-training)

前提条件：Llama-3を使用するためのアクセス

注意：これはトレーニング用のノートブックであり、推論用のノートブックは[こちら](https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b)で見つけることができます。

学びがあったり、役立つと感じた場合は、ぜひ投票してください！

# ライブラリのインポート

In [ ]:
# ライブラリのインストール
!pip install -qq peft==0.6.0  # peftライブラリのインストール
!pip install -qq bitsandbytes==0.41.1  # bitsandbytesライブラリのインストール
!pip install -qq accelerate==0.24.1  # accelerateライブラリのインストール
!pip install -qq transformers==4.35.0  # transformersライブラリのインストール
!pip install -qq torch~=2.1.0 --index-url https://download.pytorch.org/whl/cpu -q  # CPU版のPyTorchのインストール
!pip install -qq torch_xla[tpu]~=2.1.0 -f https://storage.googleapis.com/libtpu-releases/index.html -q  # TPU用のtorch_xlaのインストール
!pip uninstall -qq tensorflow -y  # TensorFlowがTPUを占有して権限エラーを引き起こすため、アンインストールします
!cp /kaggle/input/utils-xla/spmd_util.py .  # このリポジトリからのファイルコピー：https://github.com/HeegyuKim/torch-xla-SPMD

In [ ]:
import os  # osモジュールのインポート
import gc  # ガーベジコレクション用モジュールのインポート
import re  # 正規表現操作用モジュールのインポート
from time import time  # 時間関連の関数用にtimeをインポート
import random  # 乱数生成用モジュールのインポート
import warnings  # 警告メッセージの管理用モジュールのインポート
import numpy as np  # 数値計算用のライブラリNumPyのインポート
import pandas as pd  # データ操作用のライブラリPandasのインポート
import matplotlib.pyplot as plt  # グラフ描画用のmatplotlibのインポート
from tqdm.auto import tqdm  # プログレスバー表示用のtqdmをインポート

import torch  # PyTorchのインポート
import transformers  # transformersライブラリのインポート
from sklearn.metrics import accuracy_score  # 精度スコア評価のための関数インポート
from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification  # トークナイザーとモデルのインポート
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType  # Peft関連のモジュールをインポート
import torch.nn.functional as F  # PyTorchの関数型APIをインポート

import torch_xla.debug.profiler as xp  # XLAのデバッグ用プロファイラーインポート
import torch_xla.core.xla_model as xm  # XLAモデル用モジュールをインポート
import torch_xla.experimental.xla_sharding as xs  # XLAシャーディング用モジュールのインポート
import torch_xla.runtime as xr  # XLA実行時間用モジュールのインポート

xr.use_spmd()  # SPMD（Single Program Multiple Data）を使用するように設定

from torch_xla.experimental.xla_sharded_tensor import XLAShardedTensor  # XLAシャーディングされたテンソル用のインポート
from torch_xla.experimental.xla_sharding import Mesh  # XLAシャーディングのためのメッシュをインポート
from spmd_util import partition_module  # モジュールのパーティション分割用ユーティリティのインポート

tqdm.pandas()  # Pandas DataFrameとの互換性を持たせるための設定

print(f'Torch Version: {torch.__version__}')  # 現在のTorchのバージョンを表示します

In [ ]:
# ライブラリのインポート
# import os  # osモジュールのインポート
# import gc  # ガーベジコレクション用モジュールのインポート
# import re  # 正規表現操作用モジュールのインポート
# from time import time  # 時間関連の関数用にtimeをインポート
# import random  # 乱数生成用モジュールのインポート
# import warnings  # 警告メッセージの管理用モジュールのインポート
# import numpy as np  # 数値計算用のライブラリNumPyのインポート
# import pandas as pd  # データ操作用のライブラリPandasのインポート
# import matplotlib.pyplot as plt  # グラフ描画用のmatplotlibのインポート
# from tqdm.auto import tqdm  # プログレスバー表示用のtqdmをインポート

# import torch  # PyTorchのインポート
# import transformers  # transformersライブラリのインポート
# from sklearn.metrics import accuracy_score  # 精度スコア評価のための関数インポート
# from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification  # トークナイザーとモデルのインポート
# from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType  # Peft関連のモジュールをインポート
# import torch.nn.functional as F  # PyTorchの関数型APIをインポート

# import torch_xla.debug.profiler as xp  # XLAのデバッグ用プロファイラーインポート
# import torch_xla.core.xla_model as xm  # XLAモデル用モジュールをインポート
# import torch_xla.experimental.xla_sharding as xs  # XLAシャーディング用モジュールのインポート
# import torch_xla.runtime as xr  # XLA実行時間用モジュールのインポート

# xr.use_spmd()  # SPMD（Single Program Multiple Data）を使用するように設定

# from torch_xla.experimental.xla_sharded_tensor import XLAShardedTensor  # XLAシャーディングされたテンソル用のインポート
# from torch_xla.experimental.xla_sharding import Mesh  # XLAシャーディングのためのメッシュをインポート
# from spmd_util import partition_module  # モジュールのパーティション分割用ユーティリティのインポート

# tqdm.pandas()  # Pandas DataFrameとの互換性を持たせるための設定

# print(f'Torch Version: {torch.__version__}')  # 現在のTorchのバージョンを表示します
# /usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
#   from .autonotebook import tqdm as notebook_tqdm
# /usr/local/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
#   warn("The installed version of bitsandbytes was compiled without GPU support. "
# /usr/local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
# Torch Version: 2.1.2+cpu

# 設定
# class CFG:
#     NUM_EPOCHS = 1  # エポック数
#     BATCH_SIZE = 1  # バッチサイズ（以前は16）
#     DROPOUT = 0.05  # ドロップアウト率
#     MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'  # モデルのパス
#     SEED = 2024  # 乱数シード
#     MAX_LENGTH = 1024  # 最大シーケンス長
#     NUM_WARMUP_STEPS = 128  # ウォームアップステップ数
#     LR_MAX = 5e-5  # 最大学習率
#     NUM_LABELS = 3  # ラベル数
#     LORA_RANK = 4  # LoRAのランク
#     LORA_ALPHA = 8  # LoRAのアルファ値
#     LORA_MODULES = ['o_proj', 'v_proj']  # LoRAを適用するモジュール
    
# DEVICE = xm.xla_device()  # TPUデバイスの初期化

# 設定

In [ ]:
class CFG:
    NUM_EPOCHS = 1  # エポック数
    BATCH_SIZE = 1  # バッチサイズ（以前は16だった）
    DROPOUT = 0.05  # ドロップアウト率
    MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'  # モデルのパス
    SEED = 2024  # 乱数シード
    MAX_LENGTH = 1024  # 最大シーケンス長
    NUM_WARMUP_STEPS = 128  # ウォームアップステップ数
    LR_MAX = 5e-5  # 最大学習率
    NUM_LABELS = 3  # ラベルの数
    LORA_RANK = 4  # LoRAのランク
    LORA_ALPHA = 8  # LoRAのアルファ値
    LORA_MODULES = ['o_proj', 'v_proj']  # LoRAを適用するモジュールのリスト
    
DEVICE = xm.xla_device()  # TPUデバイスの初期化

In [ ]:
def set_seeds(seed):
    """再現性のためのシードを設定します"""
    os.environ['PYTHONHASHSEED'] = str(seed)  # Pythonのハッシュシードを設定
    random.seed(seed)  # randomモジュールのシードを設定
    np.random.seed(seed)  # NumPyのシードを設定
    torch.manual_seed(seed)  # PyTorchのシードを設定
    if torch.cuda.is_available():  # GPUが利用可能な場合
        torch.cuda.manual_seed(seed)  # CUDA用のシードを設定
        torch.cuda.manual_seed_all(seed)  # 全てのCUDAデバイスにシードを設定
        
    # 全TPUコアのためにシードを設定
    xm.set_rng_state(seed, device=xm.xla_device())  

set_seeds(seed=CFG.SEED)  # 設定したシードを使ってシードを設定

# トークナイザー

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG.MODEL_NAME)  # 指定したモデル名からトークナイザーをロード
tokenizer.pad_token = tokenizer.eos_token  # パディングトークンを終了トークンに設定
tokenizer.padding_side = 'right'  # パディングを右側に設定
tokenizer.add_eos_token = True  # 終了トークンを追加する設定

# 推論時にオフラインでロードできるようにトークナイザーを保存
tokenizer.save_pretrained('tokenizer')  # トークナイザーを指定したディレクトリに保存します

In [ ]:
# トークン長を取得するユーティリティ関数
def get_token_lengths(texts):
    # テキストをトークナイズし、各テキストのinput_idsを取得
    input_ids = tokenizer(texts.tolist(), return_tensors='np')['input_ids']
    # 各テキストのinput_idsの長さを返す
    return [len(t) for t in input_ids]  # 各トークンの長さをリストとして返します

# トレーニングデータの準備

In [ ]:
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')  # トレーニングデータのCSVファイルを読み込む
def process(input_str):
    stripped_str = input_str.strip('[]')  # 角括弧を取り除く
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 文章を分割し、各文章の余分な引用符を取り除く
    return ' '.join(sentences)  # 文章をスペースで結合して返す

train.loc[:, 'prompt'] = train['prompt'].apply(process)  # プロンプト列を処理
train.loc[:, 'response_a'] = train['response_a'].apply(process)  # 応答A列を処理
train.loc[:, 'response_b'] = train['response_b'].apply(process)  # 応答B列を処理

# トレーニングに向けて 'Null' を削除
indexes = train[(train.response_a == 'null') & (train.response_b == 'null')].index  # 応答Aと応答Bが両方 'null' の行を見つける
train.drop(indexes, inplace=True)  # 該当する行を削除
train.reset_index(inplace=True, drop=True)  # インデックスをリセット

print(f"総計 {len(indexes)} 行のNull応答が削除されました")
print('トレーニングサンプルの合計: ', len(train))  # 残ったトレーニングサンプルの数を表示

In [ ]:
train.head(5)  # トレーニングデータの最初の5行を表示します

In [ ]:
train['text'] = 'User prompt: ' + train['prompt'] + '\n\nModel A:\n' + train['response_a'] + '\n\n--------\n\nModel B:\n' + train['response_b']  # 各行にユーザープロンプトとモデルの応答を結合
print(train['text'][4])  # 4番目のテキストを表示します

In [ ]:
# トレーニングデータセットの50%のみを使用してトレーニング
# train = train[:int(len(train) * 0.5)]  # 最初の50%を使用する場合（コメントアウト）
train = train[int(len(train) * 0.5):]  # データセットの後半を使用

# データを2つの半分に分割
first_half = train[:int(len(train) * 0.5)]  # 最初の半分を取得

first_half.loc[:, 'token_count'] = get_token_lengths(first_half['text'])  # 最初の半分のテキストのトークン数をカウント
first_half.loc[:, 'label'] = np.argmax(first_half[['winner_model_a', 'winner_model_b', 'winner_tie']].values, axis=1)  # 勝者モデルのインデックスを取得

train.loc[:, 'token_count'] = get_token_lengths(train['text'])  # トレーニングデータ全体のトークン数をカウント

# モデルのラベルを準備し、最大インデックスを取得
train.loc[:, 'label'] = np.argmax(train[['winner_model_a', 'winner_model_b', 'winner_tie']].values, axis=1)  # 勝者モデルのインデックスを取得

# データを表示
display(train.head())  # トレーニングデータの最初の数行を表示します

In [ ]:
train.label.value_counts()  # ラベルの値カウントを表示します

In [ ]:
# トークン数の統計を表示
display(train['token_count'].describe().to_frame().astype(int))  # トークン数の統計情報を整数型のデータフレームとして表示します

In [ ]:
# データの90%をカバーするトークンの長さを取得しますが、1024の長さを維持します！
np.percentile(train['token_count'], 90)  # トークン数の90パーセンタイルを取得します

# トークナイズ

In [ ]:
# データをトークナイズします
tokens = tokenizer(
    train['text'].tolist(),  # テキストをリストとしてトークナイズ
    padding='max_length',  # 最大長でパディング
    max_length=CFG.MAX_LENGTH,  # 最大長を設定
    truncation=True,  # 長すぎるテキストは切り詰める
    return_tensors='np')  # NumPyテンソルとして返す

# 入力IDはトークンIDです
INPUT_IDS = tokens['input_ids']  # トークンIDを取得
# パディングトークンを無視するためのアテンションマスク
ATTENTION_MASKS = tokens['attention_mask']  # アテンションマスクを取得
# テキストのラベル
LABELS = train[['winner_model_a', 'winner_model_b', 'winner_tie']].values  # ラベルを取得

print(f'INPUT_IDS shape: {INPUT_IDS.shape}, ATTENTION_MASKS shape: {ATTENTION_MASKS.shape}')  # 入力IDとアテンションマスクの形状を表示
print(f'LABELS shape: {LABELS.shape}')  # ラベルの形状を表示

In [ ]:
tokens = tokenizer(
    first_half['text'].tolist(),  # 最初の半分のテキストをリストとしてトークナイズ
    padding='max_length',  # 最大長でパディングを行う
    max_length=CFG.MAX_LENGTH,  # 最大長を設定
    truncation=True,  # 長すぎるテキストは切り捨てる
    return_tensors='np'  # NumPyテンソルとして返す
)

INPUT_IDS = tokens['input_ids']  # トークンIDを取得
ATTENTION_MASKS = tokens['attention_mask']  # アテンションマスクを取得
LABELS = first_half[['winner_model_a', 'winner_model_b', 'winner_tie']].values  # ラベルを取得
print(f'INPUT_IDS shape: {INPUT_IDS.shape}, ATTENTION_MASKS shape: {ATTENTION_MASKS.shape}')  # 入力IDとアテンションマスクの形状を表示
print(f'LABELS shape: {LABELS.shape}')  # ラベルの形状を表示

In [ ]:
def train_dataset(batch_size):
    N_SAMPLES = LABELS.shape[0]  # サンプル数を取得
    IDXS = np.arange(N_SAMPLES - (N_SAMPLES % batch_size))  # バッチサイズに基づいてインデックスを作成
    while True:
        # インデックスをシャッフル
        np.random.shuffle(IDXS)
        # 全インデックスを一度だけイテレート
        for idxs in IDXS.reshape(-1, batch_size):  # バッチサイズに分割
            input_ids = torch.tensor(INPUT_IDS[idxs]).to(DEVICE)  # 入力IDをデバイスに送る
            attention_mask = torch.tensor(ATTENTION_MASKS[idxs]).to(DEVICE)  # アテンションマスクをデバイスに送る
            labels = torch.tensor(LABELS[idxs]).to(DEVICE)  # ラベルをデバイスに送る（マルチラベル出力）
            
            # TPUノードにシャーディングを行う（適用可能であれば、メッシュを適切に定義する必要があります）
            xs.mark_sharding(input_ids, mesh, (0, 1))  # 入力IDのシャーディングマーク
            xs.mark_sharding(attention_mask, mesh, (0, 1))  # アテンションマスクのシャーディングマーク
            xs.mark_sharding(labels, mesh, (0, 1))  # ラベルのシャーディングマーク
            
            yield input_ids, attention_mask, labels  # バッチを生成

TRAIN_DATASET = train_dataset(CFG.BATCH_SIZE)  # トレーニングデータセットを作成

In [ ]:
# 最初の半分のデータセット生成器を定義します
def train_dataset_first_half(batch_size):
    N_SAMPLES = LABELS.shape[0]  # サンプル数を取得
    IDXS = np.arange(N_SAMPLES - (N_SAMPLES % batch_size))  # バッチサイズに基づいてインデックスを作成
    while True:
        np.random.shuffle(IDXS)  # インデックスをシャッフル
        for idxs in IDXS.reshape(-1, batch_size):  # バッチサイズに分割
            input_ids = torch.tensor(INPUT_IDS[idxs]).to(DEVICE)  # 入力IDをデバイスに送る
            attention_mask = torch.tensor(ATTENTION_MASKS[idxs]).to(DEVICE)  # アテンションマスクをデバイスに送る
            labels = torch.tensor(LABELS[idxs]).to(DEVICE)  # ラベルをデバイスに送る
            xs.mark_sharding(input_ids, mesh, (0, 1))  # 入力IDのシャーディングマーク
            xs.mark_sharding(attention_mask, mesh, (0, 1))  # アテンションマスクのシャーディングマーク
            xs.mark_sharding(labels, mesh, (0, 1))  # ラベルのシャーディングマーク
            yield input_ids, attention_mask, labels  # バッチを生成

TRAIN_DATASET = train_dataset_first_half(CFG.BATCH_SIZE)  # 最初の半分のトレーニングデータセットを作成

# 最初の半分のためのエポックごとのステップ数を計算
STEPS_PER_EPOCH = len(first_half) // CFG.BATCH_SIZE  # エポックごとのステップ数を計算

# モデルの読み込み

In [ ]:
# 3つのターゲットラベルを用いた分類用のモデルを読み込みます
base_model = LlamaForSequenceClassification.from_pretrained(
    CFG.MODEL_NAME,  # 指定したモデル名から読み込む
    num_labels=CFG.NUM_LABELS,  # ラベルの数を指定
    torch_dtype=torch.bfloat16)  # PyTorchのデータ型を指定

base_model.config.pretraining_tp = 1  # 事前トレーニングのTPを1に設定

# パディングトークンを設定
base_model.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンIDを設定

# ローレンジ適応（LORA）

In [ ]:
lora_config = LoraConfig(
    r=CFG.LORA_RANK,  # ローランク行列の次元
    lora_alpha=CFG.LORA_ALPHA,  # LoRAのアクティベーションと事前トレーニングされた重みアクティベーションのスケーリング係数
    lora_dropout=CFG.DROPOUT,  # ドロップアウト率
    bias='none',  # バイアスの設定
    inference_mode=False,  # 推論モードの設定
    task_type=TaskType.SEQ_CLS,  # タスクタイプをシーケンス分類に設定
    target_modules=CFG.LORA_MODULES  # 出力と値のプロジェクションのみを使用
)

In [ ]:
# LoRAモデルを作成
model = get_peft_model(base_model, lora_config)  # 基本モデルとLoRA設定からLoRAモデルを生成
# 学習可能なパラメータを表示
model.print_trainable_parameters()  # 学習可能なパラメータの情報を表示します

In [ ]:
# TPUノードの数を取得
num_devices = xr.global_runtime_device_count()  # グローバルなランタイムデバイスの数を取得
mesh_shape = (1, num_devices, 1)  # メッシュの形状を定義
device_ids = np.array(range(num_devices))  # デバイスIDの配列を作成
mesh = Mesh(device_ids, mesh_shape, ('dp', 'fsdp', 'mp'))  # メッシュを作成

# モデルを分割
partition_module(model, mesh)  # モデルをメッシュに分配

print(f'num_devices: {num_devices}')  # 使用するデバイスの数を表示

In [ ]:
# 学習可能な層を確認します
MODEL_LAYERS_ROWS = []  # モデル層情報を格納するリスト
TRAINABLE_PARAMS = []  # 学習可能なパラメータを格納するリスト
N_TRAINABLE_PARAMS = 0  # 学習可能なパラメータの総数

# モデルの各パラメータをループ
for name, param in model.named_parameters():
    # レイヤーのパラメータ数を計算
    n_parameters = int(torch.prod(torch.tensor(param.shape)))
    # 学習可能な層のみ
    if param.requires_grad:
        # 層の情報を追加
        MODEL_LAYERS_ROWS.append({
            'param': n_parameters,  # パラメータ数
            'name': name,  # レイヤー名
            'dtype': param.data.dtype,  # データ型
        })
        # 学習可能なパラメータを追加
        TRAINABLE_PARAMS.append({'params': param})
        # 学習可能なパラメータの数を加算
        N_TRAINABLE_PARAMS += n_parameters
        
display(pd.DataFrame(MODEL_LAYERS_ROWS))  # モデル層情報のデータフレームを表示

print(f"""
===============================
N_TRAINABLE_PARAMS: {N_TRAINABLE_PARAMS:,}  # 学習可能なパラメータの総数を表示
N_TRAINABLE_LAYERS: {len(TRAINABLE_PARAMS)}  # 学習可能な層の数を表示
===============================
""")

# トレーニング

In [ ]:
# 学習率とオプティマイザーの設定
N_SAMPLES = len(train)  # トレーニングサンプルの数を取得
STEPS_PER_EPOCH = N_SAMPLES // CFG.BATCH_SIZE  # エポックごとのステップ数を計算

OPTIMIZER = torch.optim.AdamW(model.parameters(), lr=CFG.LR_MAX)  # AdamWオプティマイザーを設定

# ウォームアップ付きコサイン学習率スケジューラ
lr_scheduler = transformers.get_cosine_schedule_with_warmup(
    optimizer=OPTIMIZER,  # オプティマイザーを指定
    num_warmup_steps=CFG.NUM_WARMUP_STEPS,  # ウォームアップステップ数を指定
    num_training_steps=STEPS_PER_EPOCH * CFG.NUM_EPOCHS  # 総トレーニングステップ数を計算
)

print(f'BATCH_SIZE: {CFG.BATCH_SIZE}, N_SAMPLES: {N_SAMPLES}, STEPS_PER_EPOCH: {STEPS_PER_EPOCH}')  # バッチサイズ、サンプル数、エポックごとのステップ数を表示

In [ ]:
# オプティマイザーの状態（例えば、モーメンタムバッファ）のデータ型を設定
for state in OPTIMIZER.state.values():  # オプティマイザーの状態をループ
    for k, v in state.items():  # 各状態のキーと値をループ
        if isinstance(v, torch.Tensor) and state[k].dtype is not torch.float32:  # 値がテンソルで、データ型がfloat32でない場合
            state[v] = v.to(dtype=torch.float32)  # データ型をfloat32に変換

In [ ]:
input_ids, attention_mask, labels = next(TRAIN_DATASET)  # トレーニングデータセットから次のバッチを取得

print(f'input_ids shape: {input_ids.shape}, dtype: {input_ids.dtype}')  # 入力IDの形状とデータ型を表示
print(f'attention_mask shape: {attention_mask.shape}, dtype: {attention_mask.dtype}')  # アテンションマスクの形状とデータ型を表示
print(f'labels shape: {labels.shape}, dtype: {labels.dtype}')  # ラベルの形状とデータ型を表示

In [ ]:
%%time
# ダミー予測
with torch.no_grad():  # 勾配計算を行わない設定
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)  # モデルに入力IDとアテンションマスクを与えて出力を取得
    
print(f'logits: {outputs.logits}, dtype: {outputs.logits.dtype}')  # ロジットとそのデータ型を表示

In [ ]:
# モデルをトレーニングモードに設定
model.train()

# 損失関数、交差エントロピー
LOSS_FN = torch.nn.CrossEntropyLoss().to(dtype=torch.float32)  # 交差エントロピー損失関数をfloat32型に設定

In [ ]:
st = time()  # 開始時間を記録
warnings.filterwarnings("error")  # 警告をエラーとして表示
METRICS = {
    'loss': [],  # 損失を格納するリスト
    'accuracy': {'y_true': [], 'y_pred': []}  # 正解ラベルと予測ラベルを格納する辞書
}

for epoch in tqdm(range(CFG.NUM_EPOCHS)):  # エポックをループ
    ste = time()  # 各エポックの開始時間を記録
    for step in range(STEPS_PER_EPOCH):  # 各エポックのステップをループ
        # 勾配をゼロに設定
        OPTIMIZER.zero_grad()
        
        # バッチを取得
        input_ids, attention_mask, labels = next(TRAIN_DATASET)
        
        # フォワードパス
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
       
        # ロジットをfloat32に設定
        logits = outputs.logits.to(dtype=torch.float32)
        
        # バックワードパス
        loss = LOSS_FN(logits, labels.to(dtype=torch.float32))  # 損失を計算
        loss.backward()  # 勾配を計算
        
        # オプティマイザステップ
        OPTIMIZER.step()  # パラメータを更新
        xm.mark_step()  # TPUステップをマーク
        
        # 学習率スケジューラを更新
        lr_scheduler.step()
        
        # メトリクスとプログレスバーを更新
        METRICS['loss'].append(float(loss))  # 損失を追加
        METRICS['accuracy']['y_true'] += labels.squeeze().tolist()  # 正解ラベルを追加
        METRICS['accuracy']['y_pred'] += torch.argmax(F.softmax(logits, dim=-1), dim=1).cpu().tolist()  # 予測ラベルを追加
        
        if (step + 1) % 200 == 0:  # 200ステップごとにメトリクスを表示
            metrics = 'µ_loss: {:.3f}'.format(np.mean(METRICS['loss']))
            metrics += ', step_loss: {:.3f}'.format(METRICS['loss'][-1])  # 最新の損失を追加
            metrics += ', µ_auc: {:.3f}'.format(accuracy_score(torch.argmax(torch.tensor(METRICS['accuracy']['y_true']), axis=-1), \
                                                               METRICS['accuracy']['y_pred']))  # 精度を計算
            lr = OPTIMIZER.param_groups[0]['lr']  # 学習率を取得
            print(f'{epoch+1:02}/{CFG.NUM_EPOCHS:02} | {step+1:04}/{STEPS_PER_EPOCH} lr: {lr:.2E}, {metrics}', end='')
            print(f'\nSteps per epoch: {step+1} complete | Time elapsed: {time()- st}')  # 経過時間を表示
    
    print(f'\nEpoch {epoch+1} Completed | Total time for epoch: {time() - ste} ' )  # エポックの完了時間を表示

    # 停止した場合、TPUで将来的にトレーニングを続けるためにモデルとオプティマイザを保存
    xm.save({k: v.cpu() for k, v in model.named_parameters() if v.requires_grad}, f'model_llama_3_cp_{epoch+1}_v1.pth')  # モデルを保存
    xm.save(OPTIMIZER.state_dict(), f'optimizer_llama_3_cp_{epoch+1}_v1.pth')  # オプティマイザを保存
    
    print(f'Model saved at epoch {epoch+1}| Elapsed time: {time() - st} ')  # モデル保存のメッセージを表示

In [ ]:
plt.figure(figsize=(15, 6))  # グラフのサイズを設定
plt.plot(METRICS['loss'])  # 損失のプロット
plt.xlabel('Step per epoch')  # x軸のラベル
plt.ylabel('Loss')  # y軸のラベル
plt.title('Loss Plot step per epoch')  # グラフのタイトル
plt.show()  # グラフを表示

# モデルの保存

In [ ]:
model = model.cpu()  # モデルをCPUに移動
torch.save(dict([(k, v) for k, v in model.named_parameters() if v.requires_grad]), 'llama_3_finetuned_model.pth')  # 学習可能なパラメータを保存

# 結論

トレーニングの速度を上げて最適化する余地はまだたくさんあります！ より多くのデータや異なるバッチサイズ、学習率を試してみてください... すべての成功を祈ります！